# [Paris Saclay Center for Data Science](http://www.datascience-paris-saclay.fr)


# Fraud detection


## Introduction


### Requirements

* numpy>=1.10.0  
* matplotlib>=1.5.0 
* pandas>=0.19.0  
* scikit-learn>=0.19   

In [1]:
%matplotlib inline
import os
import glob
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_columns = 999

## Exploratory data analysis

### Loading the data



In [2]:
train_filename = 'kaggle_data/train.csv'

In [3]:
data = pd.read_csv(train_filename)

In [4]:
data.head()

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,4273829,307,CASH_IN,69485.96,C1128681951,4728013.74,4797499.71,C1837241711,2079611.18,2010125.22,0,0
1,5070895,355,CASH_OUT,272619.50,C1750753248,25341.00,0.00,C540691118,190274.43,462893.93,0,0
2,5138032,356,CASH_OUT,261840.00,C945986755,0.00,0.00,C338984620,692171.14,954011.14,0,0
3,831114,41,CASH_OUT,17743.62,C1653894687,273190.00,255446.38,C689874395,0.00,17743.62,0,0
4,4710133,332,CASH_IN,144781.34,C1199680151,3126158.06,3270939.40,C518951914,227662.41,82881.07,0,0


In [5]:
data.describe()

,id,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,5.090096e+06,5.090096e+06,5.090096e+06,5.090096e+06,5.090096e+06,5.090096e+06,5.090096e+06,5.090096e+06,5.090096e+06
mean,3.180916e+06,2.433690e+02,1.798795e+05,8.342038e+05,8.554146e+05,1.100630e+06,1.224822e+06,1.287402e-03,2.750439e-06
std,1.836845e+06,1.423376e+02,6.053800e+05,2.887990e+06,2.923804e+06,3.414843e+06,3.689388e+06,3.585729e-02,1.658443e-03
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.590031e+06,1.550000e+02,1.337681e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.180492e+06,2.390000e+02,7.474478e+04,1.420456e+04,0.000000e+00,1.324753e+05,2.142185e+05,0.000000e+00,0.000000e+00
75%,4.771922e+06,3.350000e+02,2.085979e+05,1.073850e+05,1.444491e+05,9.420183e+05,1.110987e+06,0.000000e+00,0.000000e+00
max,6.362619e+06,7.430000e+02,7.382349e+07,5.731626e+07,4.731626e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5090096 entries, 0 to 5090095
Data columns (total 12 columns):
id                int64
step              int64
type              object
amount            float64
nameOrig          object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest          object
oldbalanceDest    float64
newbalanceDest    float64
isFraud           int64
isFlaggedFraud    int64
dtypes: float64(5), int64(4), object(3)
memory usage: 466.0+ MB


In [7]:
data.count()

id                5090096
step              5090096
type              5090096
amount            5090096
nameOrig          5090096
oldbalanceOrg     5090096
newbalanceOrig    5090096
nameDest          5090096
oldbalanceDest    5090096
newbalanceDest    5090096
isFraud           5090096
isFlaggedFraud    5090096
dtype: int64

In [8]:
np.unique(data['isFraud'])

array([0, 1])

In [9]:
data.groupby('isFraud').count()[['id']]

,id
isFraud,
0,5083543
1,6553


## The pipeline

For submitting at the [RAMP site](http://ramp.studio), you will have to write two classes, saved in two different files,
* the class `FeatureExtractor`, which will be used to extract features for classification from the dataset and produce a numpy array of size (number of samples $\times$ number of features), and  
* the class `Classifier` to predict the target.

### Feature extractor

The feature extractor implements a `transform` member function. It is saved in the file [`submissions/starting_kit/feature_extractor.py`](/edit/submissions/starting_kit/feature_extractor.py). It receives the pandas dataframe `X_df` defined at the beginning of the notebook. It should produce a numpy array representing the extracted features, which will then be used for the classification.  

Note that the following code cells are *not* executed in the notebook. The notebook saves their contents in the file specified in the first line of the cell, so you can edit your submission before running the local test below and submitting it at the RAMP site.

In [10]:
%%file submissions/starting_kit/feature_extractor.py
class FeatureExtractor():
    def __init__(self):
        pass

    def fit(self, X_df, y):
        pass

    def transform(self, X_df):
        X_df = X_df.drop(['nameDest','nameOrig','type'],axis = 1)
        return X_df.values

Overwriting submissions/starting_kit/feature_extractor.py


### Classifier

The classifier follows a classical scikit-learn classifier template. It should be saved in the file [`submissions/starting_kit/classifier.py`](/edit/submissions/starting_kit/classifier.py). In its simplest form it takes a scikit-learn pipeline, assigns it to `self.clf` in `__init__`, then calls its `fit` and `predict_proba` functions in the corresponding member funtions.

In [11]:
%%file submissions/starting_kit/classifier.py
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier


class Classifier(BaseEstimator):
    def __init__(self):
        pass

    def fit(self, X, y):
        self.clf = RandomForestClassifier(
            n_estimators=2, max_leaf_nodes=2, random_state=61)
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)

Overwriting submissions/starting_kit/classifier.py


## Local testing (before submission)

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. For this we provide a unit test. Note that the test runs on your files in [`submissions/starting_kit`](/tree/submissions/starting_kit), not on the classes defined in the cells of this notebook.

First `pip install ramp-workflow` or install it from the [github repo](https://github.com/paris-saclay-cds/ramp-workflow). Make sure that the python files `classifier.py` and `feature_extractor.py` are in the  [`submissions/starting_kit`](/tree/submissions/starting_kit) folder, and the data `train.csv` and `test.csv` are in [`data`](/tree/data). Then run

```ramp_test_submission```

If it runs and print training and test errors on each fold, then you can submit the code.


In [12]:
#!ramp_test_submission

You can use the `--quick-test` switch to test the notebook on the mock data sets in `data/`. Since the data is random, the scores will not be meaningful, but it can be useful to run this first on your submissions to make sure they run without errors.

In [13]:
!ramp_test_submission --quick-test

Testing Fraud detection
Reading train and test files from ./data ...
Reading cv ...
Training ./submissions/starting_kit ...
CV fold 0
	score  kappa  matthews  roc_auc    acc
	train    0.0       0.0    0.543  0.999
	valid    0.0       0.0    0.525  0.999
	test     0.0       0.0    0.499  1.000
CV fold 1
	score  kappa  matthews  roc_auc    acc
	train    0.0       0.0    0.817  0.999
	valid    0.0       0.0    0.754  0.999
	test     0.0       0.0    0.379  1.000
CV fold 2
	score  kappa  matthews  roc_auc    acc
	train  0.034     0.132    0.758  0.999
	valid -0.000    -0.000    0.791  0.999
	test  -0.000    -0.000    0.369  1.000
CV fold 3
	score  kappa  matthews  roc_auc    acc
	train  0.034     0.132    0.747  0.999
	valid  0.000     0.000    0.720  0.999
	test   0.000     0.000    0.412  1.000
CV fold 4
	score  kappa  matthews  roc_auc    acc
	train    0.0       0.0    0.802  0.999
	valid    0.0       0.0    0.746  0.999
	test     0.0       0.0    0.390  1.000
CV fold 5
	score  kappa  m

## Other models in the starting kit

You can also keep several other submissions in your work directory [`submissions`](/tree/submissions) and test them using
```
ramp_test_submission --submission <submission_name>
```
where `<submission_name>` is the name of the folder in `submissions/`.

## Submitting to [ramp.studio](http://ramp.studio)

If you are eligible, you can join the team at [ramp.studio](http://www.ramp.studio). If it is your first time using RAMP, [sign up](http://www.ramp.studio/sign_up), otherwise [log in](http://www.ramp.studio/login). 

Once your signup request is accepted, you can go to your [sandbox](http://www.ramp.studio/events/kaggle_seguro/sandbox) and copy-paste (or upload) [`feature_extractor.py`](/edit/submissions/starting_kit/feature_extractor.py) and [`classifier.py`](/edit/submissions/starting_kit/classifier.py) from `submissions/starting_kit`. Save it, rename it, then submit it. The submission is trained and tested on our backend in the same way as `ramp_test_submission` does it locally. While your submission is waiting in the queue and being trained, you can find it in the "New submissions (pending training)" table in [my submissions](http://www.ramp.studio/events/kaggle_seguro/my_submissions). Once it is trained, you get a mail, and your submission shows up on the [public leaderboard](http://www.ramp.studio/events/kaggle_seguro/leaderboard). 
If there is an error (despite having tested your submission locally with `ramp_test_submission`), it will show up in the "Failed submissions" table in [my submissions](http://www.ramp.studio/events/kaggle_seguro/my_submissions). You can click on the error to see part of the trace.

After submission, do not forget to give credits to the previous submissions you reused or integrated into your submission.

The data set we use at the backend is usually different from what you find in the starting kit, so the score may be different.

The usual way to work with RAMP is to explore solutions, add feature transformations, select models, perhaps do some AutoML/hyperopt, etc., _locally_, and checking them with `ramp_test_submission`. The script prints mean cross-validation and test scores 
```
----------------------------
train ngini = 0.119 ± 0.007
train auc = 0.559 ± 0.003
train acc = 0.964 ± 0.0
train nll = 0.156 ± 0.0
valid ngini = 0.114 ± 0.005
valid auc = 0.558 ± 0.002
valid acc = 0.964 ± 0.0
valid nll = 0.156 ± 0.0
test ngini = 0.229 ± 0.256
test auc = 0.307 ± 0.064
test acc = 1.0 ± 0.0
test nll = 0.037 ± 0.0
```
and bagged cross-validation and test scores
```
valid ngini = 0.167
test ngini = -0.324
```
This latter combines the cross-validation models pointwise on the validation and test sets, and usually leads to a better score than the mean CV score. The RAMP [leaderboard](http://www.ramp.studio/events/kaggle_seguro/leaderboard) displays this score.

The official score in this RAMP (the first score column after "historical contributivity" on the [leaderboard](http://www.ramp.studio/events/kaggle_seguro/leaderboard)) is normalized Gini ("ngini"), so the line that is relevant in the output of `ramp_test_submission` is `valid ngini = 0.167`. When the score is good enough, you can submit it at the RAMP.

## More information

You can find more information in the [README](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/README.md) of the [ramp-workflow library](https://github.com/paris-saclay-cds/ramp-workflow).

## Contact

Don't hesitate to [contact us](mailto:admin@ramp.studio?subject=kaggle seguro notebook).